In [32]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [38]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [34]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)
df.show()


23/04/04 12:26:17 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137

In [43]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [35]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales')

In [58]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql('SELECT date_built, round(avg(price),2) AS average_price FROM HOME_SALES WHERE bedrooms == 4 GROUP BY 1').show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|    307908.86|
|      2013|    299999.39|
|      2014|    299073.89|
|      2012|    298233.42|
|      2016|    296050.24|
|      2010|    296800.75|
|      2011|     302141.9|
|      2017|    296576.69|
+----------+-------------+



In [57]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql('SELECT date_built, round(avg(price),2) AS average_price FROM HOME_SALES  WHERE bedrooms == 3 AND BATHROOMS = 3 GROUP BY 1').show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|     288770.3|
|      2013|    295962.27|
|      2014|    290852.27|
|      2012|    293683.19|
|      2016|    290555.07|
|      2010|    292859.62|
|      2011|    291117.47|
|      2017|    292676.79|
+----------+-------------+



In [63]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql('SELECT date_built, \
    round(avg(price),2) AS average_price \
    FROM HOME_SALES  \
    WHERE bedrooms == 3 AND BATHROOMS == 3 AND FLOORS == 2 AND SQFT_LIVING >= 2000 \
    GROUP BY 1').show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|    297609.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2012|    307539.97|
|      2016|     293965.1|
|      2010|    285010.22|
|      2011|    276553.81|
|      2017|    280317.58|
+----------+-------------+



In [64]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql('SELECT VIEW, \
    round(avg(price),2) AS average_price \
    FROM HOME_SALES  \
    WHERE PRICE >= 350000 \
    GROUP BY 1').show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|VIEW|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.16062498092651367 seconds ---


In [65]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [66]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [67]:
# 9. Using the cached data, run the query that filters out the view ratings with average price of greater than $350,000. 
# Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql('SELECT VIEW, \
    round(avg(price),2) AS average_price \
    FROM HOME_SALES  \
    WHERE PRICE >= 350000 \
    GROUP BY 1').show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|VIEW|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.13854598999023438 seconds ---


In [68]:
# 10. Partition the home sales dataset by the date_built field.
df.write.parquet('parquet_home_sales', mode='overwrite')

In [69]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet('parquet_home_sales')

In [70]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [71]:
# 13. Run the query that filters out the view ratings with average price of greater than $350,000 with the parquet DataFrame. 
# Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

spark.sql('SELECT VIEW, \
    round(avg(price),2) AS average_price \
    FROM HOME_SALES  \
    WHERE PRICE >= 350000 \
    GROUP BY 1').show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|VIEW|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.07796716690063477 seconds ---


In [72]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [73]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False